In [ ]:
import os
import skimage
import skimage.feature
import pydicom
from matplotlib import pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

dirname = "Data/Reg" #npy files dir name

In [ ]:
filenames = [el for el in os.listdir(dirname) if el.endswith('.npy')]
dataset = np.zeros((len(filenames), 486), dtype=np.float32)
for i, filename in enumerate(filenames):
    img = np.load(os.path.join(dirname, filename))[:,:, 0]
    sobel_img = skimage.filters.sobel(img)
    hog, hog_image = skimage.feature.hog(skimage.transform.rescale(sobel_img, 1/11), orientations=9, visualize=True, multichannel = False)
    dataset[i] = hog

# PCA

In [ ]:
pca = PCA(n_components=26)
pca.fit(dataset)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
pca_dataset = pca.transform(dataset)

# KMeans Clustering

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(pca_dataset)

In [ ]:
kmeans.labels_

In [ ]:
import _pickle as pickle
pickle.dump(kmeans, open("model.pkl", 'wb'))

In [ ]:
for i, filename in enumerate(filenames):
    if not "rev" in filename:
        print("Label: ", kmeans.labels_[i])
        img = np.load(dirname + "/" + filename)
        ogfile = pydicom.read_file("Data/Raw/" + filename[:][:-4] + ".dcm")
        plt.imshow(ogfile.pixel_array)
        plt.show()
        plt.imshow(img)
        plt.show()

# Move and Load Files

In [ ]:
for i, filename in enumerate(filenames):
    os.rename(os.path.join(dirname, filename), os.path.join("Data", str(kmeans.labels_[i]), filename))

In [ ]:
for filename in os.listdir("Data/1"):
    if not "rev" in filename:
        img = np.load("Data/1/" + filename)
        ogfile = pydicom.read_file("Data/Raw/" + filename[:][:-4] + ".dcm")
        plt.imshow(ogfile.pixel_array)
        plt.show()

In [ ]:
img = np.load(dirname + "/" + "422.npy")
ogfile = pd.read_file("Data/Raw/" + "422.dcm")
plt.imshow(ogfile.pixel_array)
plt.show()
plt.imshow(img)
plt.show()